# Databricks Autoloader Ingest

Placeholder notebook. Use Databricks Autoloader to ingest employment contracts from Blob Storage into Delta Bronze.

In [ ]:
from pyspark.sql.functions import current_timestamp

raw_path = "abfss://contracts@ragstorage4122025.dfs.core.windows.net/"
bronze_path = "abfss://bronze@ragstorage4122025.dfs.core.windows.net/contracts_bronze/"

# Read all raw files as binary
bronze_df = (
    spark.read.format("binaryFile")
    .load(raw_path)
    # binaryFile gives: path, modificationTime, length, content
    .withColumnRenamed("path", "filename")
    .withColumn("ingest_timestamp", current_timestamp())
)

display(bronze_df)  # ✅ You should see one row per file

bronze_df.write.format("delta").mode("overwrite").save(bronze_path)